In [1]:
from pathlib import Path


In [2]:
models = '''# generated by fastapi-codegen:
#   filename:  openapi.json
#   timestamp: 2024-04-29T20:24:14+00:00

from __future__ import annotations

from typing import List, Optional, Union

from pydantic import BaseModel, Field


class Item(BaseModel):
    name: str = Field(..., description='Name of the item', title='Name')
    description: str = Field(
        ..., description='Description of the item', title='Description'
    )
    price: float = Field(..., description='Price of the item', title='Price')
    tax: float = Field(..., description='Tax of the item', title='Tax')


class ValidationError(BaseModel):
    loc: List[Union[str, int]] = Field(..., title='Location')
    msg: str = Field(..., title='Message')
    type: str = Field(..., title='Error Type')


class HTTPValidationError(BaseModel):
    detail: Optional[List[ValidationError]] = Field(None, title='Detail')
'''

exec(models)

In [3]:
import requests
import inspect
from functools import wraps
import re
from typing import Any, Callable, List, Optional, Tuple
from unittest.mock import MagicMock

# proxy = MagicMock()
# Proxy = MagicMock(return_value=proxy)
# proxy.put = MagicMock()

def get_params(path: str, func: Callable[..., Any ]) -> Tuple[List[str], List[str], Optional[str]]:
    sig = inspect.signature(func)

    params_names = set(sig.parameters.keys())
    # print(f"{params_names=}")

    path_params = set(re.findall(r'\{(.+?)\}', path))
    # print(f"{path_params=}")
    if not path_params.issubset(params_names):
        raise ValueError(f"Path params {path_params} not in {params_names}")
    
    body = "body" if "body" in params_names else None

    q_params = set(params_names) - path_params - {body}

    return q_params, path_params, body

class Proxy:
    def __init__(self, servers, **kwargs):
        self.servers = servers
        self.kwargs = kwargs

    def put(self, path, **kwargs):
        def decorator(func):
            q_params, path_params, body = get_params(path, func)

            @wraps(func)
            def wrapper(*args, **kwargs):
                expanded_path = path.format(**kwargs)

                url = self.servers[0]["url"] + path.format(**{p: kwargs[p] for p in path_params})
                
                body_dict = {
                    "json": kwargs[body].model_dump() if hasattr(kwargs[body], "model_dump") else kwargs[body].dict(),
                    "headers": {'Content-Type': 'application/json'},
                } if body else {}

                params={k: v for k,v in kwargs.items() if k in q_params}

                print(f"{url=}")
                print(f"{params=}")
                print(f"{body_dict=}")
                response = requests.put(
                    url,
                    params=params,
                    **body_dict
                )

                return response.json()


            return wrapper
        
        return decorator

In [4]:
main = '''# generated by fastapi-codegen:
#   filename:  openapi.json
#   timestamp: 2024-04-29T20:24:14+00:00

from __future__ import annotations

from typing import Any, Optional, Union

# from fastapi import FastAPI

# from models import HTTPValidationError, Item

app = Proxy(
    title='FastAPI',
    version='0.1.0',
    servers=[{'url': 'http://localhost:8080', 'description': 'Local environment'}],
)


@app.put(
    '/items/{item_id}/ships/{ship}',
    response_model=Any,
    responses={'422': {'model': HTTPValidationError}},
)
def update_item_items__item_id__ships__ship__put(
    item_id: int,
    ship: str = ...,
    q1: Optional[str] = None,
    q2: Optional[int] = None,
    body: Item = ...,
) -> Union[Any, HTTPValidationError]:
    """
    Update Item
    """
    pass

'''

exec(main)

In [5]:
update_item_items__item_id__ships__ship__put(item_id=1, ship="Marry Jane", q1="q1", q2=2, body=Item(name="name", description="description", price=1.0, tax=2.0))

url='http://localhost:8080/items/1/ships/Marry Jane'
params={'q1': 'q1', 'q2': 2}
body_dict={'json': {'name': 'name', 'description': 'description', 'price': 1.0, 'tax': 2.0}, 'headers': {'Content-Type': 'application/json'}}


ConnectionError: HTTPConnectionPool(host='localhost', port=8080): Max retries exceeded with url: /items/1/ships/Marry%20Jane?q1=q1&q2=2 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x76b234c45c40>: Failed to establish a new connection: [Errno 111] Connection refused'))